# Notebook for Computational Graphs

In [1]:
import Pkg
Pkg.activate(".")
Pkg.status()

  Activating project at `~/VerifyNN/DPNeurifyFV.jl`


Project DPNeurifyFV v0.1.0
Status `~/VerifyNN/DPNeurifyFV.jl/Project.toml`
  [336ed68f] CSV v0.10.11
  [8f4d0f93] Conda v1.9.0
  [864edb3b] DataStructures v0.18.13
  [587475ba] Flux v0.13.17
  [b4f0291d] LazySets v2.7.6
⌅ [6405355b] Mosek v1.2.3
  [7e1232c4] NeuralPriorityOptimizer v0.1.0 `https://github.com/sisl/NeuralPriorityOptimizer.jl#main`
  [146f25fa] NeuralVerification v0.1.0 `https://github.com/sisl/NeuralVerification.jl#0d9be34`
  [070f6fb0] OnnxReader v0.1.0 `https://github.com/phK3/OnnxReader.jl#master`
  [d96e819e] Parameters v0.12.3
  [295af30f] Revise v3.5.3
  [a2eb1b4f] VnnlibParser v0.1.0 `https://github.com/phK3/VnnlibParser.jl#master`
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌅ have new versions available but cannot be upgraded. To see why use `status --outdated`


In [2]:
using Revise, NeuralVerification, DPNeurifyFV, LazySets, Flux
const NV = NeuralVerification
const DP = DPNeurifyFV

Academic license - for non-commercial use only - expires 2023-11-26


  ** incremental compilation may be fatally broken for this module **



Academic license - for non-commercial use only - expires 2023-11-26


DPNeurifyFV

In [3]:
input_set, output_set = DP.get_acas_sets(1)
acas = read_nnet("./networks/ACASXU_experimental_v2a_1_1.nnet")
acas_npi = NetworkNegPosIdx(acas);

In [4]:
layer1 = DP.Linear(String[], ["relu1"], "linear1", acas.layers[1].weights, acas.layers[1].bias)

# last layer only has id activation
relu_nodes = [DP.Relu(["linear$i"], ["linear$(i+1)"], "relu$i") for i in 1:length(acas.layers)-1]
linear_nodes = [DP.Linear(["relu$(i-1)"], ["relu$i"], "linear$i", acas.layers[i].weights, acas.layers[i].bias) for i in 2:length(acas.layers)-1]

layer7 = DP.Linear(["relu6"], String[], "linear7", acas.layers[7].weights, acas.layers[7].bias)

nodes = [layer1, layer7, relu_nodes..., linear_nodes...]

13-element Vector{DPNeurifyFV.Node}:
 DPNeurifyFV.Linear(String[], ["relu1"], "linear1", Dense(5 => 50), Dense(5 => 50), Dense(5 => 50))
 DPNeurifyFV.Linear(["relu6"], String[], "linear7", Dense(50 => 5), Dense(50 => 5), Dense(50 => 5))
 DPNeurifyFV.Relu(["linear1"], ["linear2"], "relu1")
 DPNeurifyFV.Relu(["linear2"], ["linear3"], "relu2")
 DPNeurifyFV.Relu(["linear3"], ["linear4"], "relu3")
 DPNeurifyFV.Relu(["linear4"], ["linear5"], "relu4")
 DPNeurifyFV.Relu(["linear5"], ["linear6"], "relu5")
 DPNeurifyFV.Relu(["linear6"], ["linear7"], "relu6")
 DPNeurifyFV.Linear(["relu1"], ["relu2"], "linear2", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50))
 DPNeurifyFV.Linear(["relu2"], ["relu3"], "linear3", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50))
 DPNeurifyFV.Linear(["relu3"], ["relu4"], "linear4", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50))
 DPNeurifyFV.Linear(["relu4"], ["relu5"], "linear5", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50))
 DPNeurifyFV.Linear(["relu5"]

In [5]:
nn = DP.CompGraph(nodes, layer1, layer7)

DPNeurifyFV.CompGraph(Dict{Any, Any}("linear6" => DPNeurifyFV.Linear(["relu5"], ["relu6"], "linear6", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50)), "linear4" => DPNeurifyFV.Linear(["relu3"], ["relu4"], "linear4", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50)), "linear3" => DPNeurifyFV.Linear(["relu2"], ["relu3"], "linear3", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50)), "relu1" => DPNeurifyFV.Relu(["linear1"], ["linear2"], "relu1"), "relu6" => DPNeurifyFV.Relu(["linear6"], ["linear7"], "relu6"), "relu5" => DPNeurifyFV.Relu(["linear5"], ["linear6"], "relu5"), "linear1" => DPNeurifyFV.Linear(String[], ["relu1"], "linear1", Dense(5 => 50), Dense(5 => 50), Dense(5 => 50)), "relu2" => DPNeurifyFV.Relu(["linear2"], ["linear3"], "relu2"), "relu4" => DPNeurifyFV.Relu(["linear4"], ["linear5"], "relu4"), "linear2" => DPNeurifyFV.Linear(["relu1"], ["relu2"], "linear2", Dense(50 => 50), Dense(50 => 50), Dense(50 => 50))…), DPNeurifyFV.Linear(String[], ["relu1"], "linear1", Dense(5 =

In [6]:
x = zeros(5)
y_graph = DP.propagate(nn, x)
y_nnet  = DP.compute_output(acas, x)

@show y_graph
@show y_nnet;

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(5 => 50)
│   summary(x) = 5-element Vector{Float64}
└ @ Flux /home/philipp/.julia/packages/Flux/n3cOc/src/layers/stateless.jl:60


y_graph = Float32[-0.021198861, -0.018714212, -0.01876629, -0.018762132, -0.018760461]
y_nnet = 

[-0.021198862641387452, -0.018714213200079682, -0.018766289570347476, -0.018762131256868687, -0.01876046090390901]


In [7]:
s = init_symbolic_interval_fvheur(acas_npi, input_set, max_vars=0)
ŝ = forward_network(DPNFV(method=:DeepPolyRelax), acas_npi, s)
ŝs = forward_network(DPNFV(method=:DeepPolyRelax), acas_npi, s, collect=true)
ŝs_before = forward_network(DPNFV(method=:DeepPolyRelax), acas_npi, s, collect=true, before_act=true)

8-element Vector{DPNeurifyFV.SymbolicIntervalFVHeur{Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}, Float64}}:
 DPNeurifyFV.SymbolicIntervalFVHeur{Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}, Float64}([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0], Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([0.63992888435, 0.0, 0.0, 0.475, -0.475], [0.03992888435000008, 0.5, 0.5, 0.025000000000000022, 0.024999999999999967]), [[-1.0971559800000001, -0.7980498639989381, -0.5797057, -1.4966126666463049, -0.5463720289717371, -2.1905491758558364, 0.32656089999999993, -1.376062546037025, -1.523329208616126, -0.94488313993238  …  -0.4998068, 0.48509536999999997, 0.24354484999999992, -0.7422393959963329, -0.6970228708495904, -0.5706770931557293, -0.19902163, -0.42158674053592626, -1.5677061648794974, 0.7375468], [-2.6693529392464455, -0.5026599879163651, -4

In [8]:
s_graph = DP.init_symbolic_interval_graph(nn, input_set, max_vars=0)
ŝ_graph = DP.propagate(DPNFV(method=:DeepPolyRelax), nn, s_graph)
graph_dict = DP.propagate(DPNFV(method=:DeepPolyRelax), nn, s_graph, return_dict=true)

Dict{String, DPNeurifyFV.SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}, Float64}} with 13 entries:
  "linear6" => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "linear4" => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "linear3" => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "relu1"   => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "relu6"   => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "relu5"   => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "linear1" => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "relu2"   => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "linear2" => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "relu4"   => SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, V…
  "linear7" => SymbolicIntervalGraph{Hyperrectangle{Float

In [9]:
ll, lu = DP.bounds(ŝ.Low, input_set)
ul, uu = DP.bounds(ŝ.Up,  input_set)

@show ll
@show uu;

ll = [-945.7437378443677, -1593.129272079237, -1107.3275048832113, -2657.431833411753, -1721.5226671840164]
uu = [2627.2788360915083, 3430.9961003041044, 3485.8259676663456, 3830.7907683913636, 3815.8806839777667]


In [10]:
ll_graph, lu_graph = DP.bounds(ŝ_graph.Low, input_set)
ul_graph, uu_graph = DP.bounds(ŝ_graph.Up, input_set)

@show ll_graph
@show uu_graph;

ll_graph = [-945.7437688112259, -1593.1292766571046, -1107.3275573968888, -2657.4320477485658, -1721.5226411342621]
uu_graph = [2627.278978729248, 3430.996284389496, 3485.826116275787, 3830.7908802986144, 3815.8809765815736]


In [11]:
# sn = DP.SplitNode(["parent"], ["child"], "split0", splits=[2, 3])
sn = DP.SplitNode(["parent"], ["child"], "split0", num_outputs=2)

DPNeurifyFV.SplitNode(["parent"], ["child"], "split0", 1, nothing, 2)

In [13]:
starts = [0; cumsum(sn.splits)]
stops = cumsum(sn.splits)
ranges = [start+1:stop for (start,stop) in zip(starts, stops)]

2-element Vector{UnitRange{Int64}}:
 1:2
 3:5

In [24]:
idxs = [1:size(ŝ_graph, i) for i in 1:ndims(ŝ_graph)]
idxs[1] = ranges[1]
ŝ_graph[idxs...]

DPNeurifyFV.SymbolicIntervalGraph{Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}, Float64}(Float32[63.50458 64.99495 … 52.03738 -722.34534; 107.78185 109.22323 … 84.61534 -1218.1754], Float32[-178.78766 -174.02061 … -139.2767 2012.038; -233.6232 -227.37111 … -181.14955 2628.048], Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([0.63992888435, 0.0, 0.0, 0.475, -0.475], [0.03992888435000008, 0.5, 0.5, 0.025000000000000022, 0.024999999999999967]), Dict{Any, Any}(), Dict{Any, Any}(), Matrix{Float64}(undef, 0, 6), Matrix{Float64}(undef, 0, 6), Any[], 0, [87986.68430948257, 85363.30530929565, 572342.4810791016, 9143.81452178955, 70544.162545681])

In [13]:
DP.forward_node(DPNFV(), sn, ŝ_graph)

In [16]:
ŝ_graph[ranges[1]].Low

2×6 Matrix{Float32}:
  63.5046   64.9949  -411.852   6.5352  52.0374   -722.345
 107.782   109.223   -695.48   11.1919  84.6153  -1218.18

In [23]:
DP.forward_node(DPNFV(), sn, ŝ_graph)

In [15]:
function Base.getindex(s::DP.SymbolicIntervalGraph, inds...)
    # TODO: what about max_vars (should there be less vars for the split interval?)
    # and var_ids (should we truncate zero entries?)
    # only use indices for first dimensions, last dimension is for coefficients
    Low = s.Low[inds...,:]
    Up  = s.Up[inds...,:]
    return DP.init_symbolic_interval_graph(s, Low, Up)
end

In [40]:
function Base.ndims(s::DP.SymbolicIntervalGraph)
    return length(dim(s))
end

In [27]:
function Base.size(s::DP.SymbolicIntervalGraph, d)
    return dim(s)[d]
end

In [41]:
ndims(ŝ_graph)

1

In [21]:
ŝ_graph[1:1].Low

1×6 Matrix{Float32}:
 63.5046  64.9949  -411.852  6.5352  52.0374  -722.345

In [39]:
size(ŝ_graph, 5)

BoundsError: BoundsError: attempt to access Tuple{Int64} at index [5]

In [35]:
methods(size(AbstractArray))

MethodError: MethodError: no method matching size(::Type{AbstractArray})
Closest candidates are:
  size(!Matched::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}) at /opt/julia-1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/qr.jl:581
  size(!Matched::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted}, !Matched::Integer) at /opt/julia-1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/qr.jl:580
  size(!Matched::Union{LinearAlgebra.Cholesky, LinearAlgebra.CholeskyPivoted}) at /opt/julia-1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/cholesky.jl:514
  ...

In [32]:
methods(Base.size(DP.SymbolicIntervalGraph, Integer))

MethodError: MethodError: no method matching size(::Type{DPNeurifyFV.SymbolicIntervalGraph}, ::Type{Integer})
Closest candidates are:
  size(!Matched::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}, ::Any) at /opt/julia-1.8.0/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:203
  size(!Matched::Union{HDF5.HDF5Attribute, HDF5.HDF5Dataset}, ::Any) at ~/.julia/packages/HDF5/T1b9x/src/HDF5.jl:1089
  size(!Matched::Zygote.Buffer, ::Any...; kwargs...) at ~/.julia/packages/MacroTools/qijNY/src/examples/forward.jl:17
  ...